
## Input File Requirement to Generate Publications:
1. People ID (Row number recommended)
2. First Name
3. Last Name
4. Searched Term
5. Extra Information(Any Columns)

## Remember:
#####  The only needed variables are the first 4. The rest variables will be kept as the input for later comparation.

## ReadMe!!!!, Update Description:
1. Now the input name and searched term are auto-collected to the ouput
2. Now it is a Full name Matching Algorithm!!!!!!!!
3. Latin Name, Letter Case won't affect the Name Matching
4. A lot of useless columns droped.
5. Any amounts of Extra Information are available now!!!!!!!!!!!!!!
6. All the parameters need to be modified are in the first cell now.


7. Similar name will be collected. The DS_Rank will be assigned.
8. Empty name now won't cause error.


In [1]:
# import necessary libraries
# Entrez is one library adapting the E-Utilities functions of Pubmed search for Python
# Note: no more than 3 queries/second using E-Utilities or more than 100,000 results /
# query -- will require setting up API Key from Pubmed
from Bio import Entrez
import pprint
import numpy as np
import pandas as pd
from datetime import datetime
import os
import re
import unidecode as un
# Register the API in PubMed!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Entrez.email = "juan.lin@h1insights.com"
Entrez.api_key = "1dbfc1e1be64af19af3d7f66d331825d0409"


# Set up your directory
os.chdir('C:/Users/Juan/Desktop/ipython')
df = pd.read_excel("HKfull.xlsx", index_col=None)

file_name = "HKfull"

searchterm = "search='({} {}[Author]) AND ({}[Affiliation])'.format(authorfirst, authorlast, org)"
#searchterm = "search='{} {} {}'.format(authorfirst, authorlast, org)"
#searchterm = "search='{} {}[Author]'.format(authorfirst, authorlast)"

In [2]:
# from fuzzywuzzy import fuzz
# fuzz.ratio("this is a test", "this is a test!")
def difference(first, second):
    ''' 编辑距离算法（LevD） 
        Args: 两个字符串
        returns: 两个字符串的编辑距离 int
    '''
    if len(first) > len(second):
        first, second = second, first
    if len(first) == 0:
        return len(second)
    if len(second) == 0:
        return len(first)
    first_length = len(first) + 1
    second_length = len(second) + 1
    distance_matrix = [list(range(second_length)) for x in range(first_length)]
    # print distance_matrix
    for i in range(1, first_length):
        for j in range(1, second_length):
            deletion = distance_matrix[i - 1][j] + 1
            insertion = distance_matrix[i][j - 1] + 1
            substitution = distance_matrix[i - 1][j - 1]
            if first[i - 1] != second[j - 1]:
                substitution += 1
            distance_matrix[i][j] = min(insertion, deletion, substitution)
            # print distance_matrix
    return distance_matrix[first_length - 1][second_length - 1]


# function to generate dataframe representing Pubmed search for one person with article information:\
# Name, Organization Affiliation, Title of Paper, DOI
def generator(PMID, record, shape):
    # initiate dictionaries for storing the different columns of information, will at end convert dictionary to series to pandas dataframe
    # dictionary adds additional security because keeps PMID value for the article information we're seeking
    #dicts = {}
    dicts1 = {}
    dicts2 = {}
    dicts3 = {}
    title = []  # article title - /ArticleTitle
    peopleID = []                        #

    matchto = []
    affiliation = []
    name = []

    DS_rank = []

    # iterates through the individual articles
    # length of pubmed articles in records (not PMID, because this will not include book articles (included in records at very end))
    for i in range(len(record['PubmedArticle'])):
        peopleID.append(ID)
        matchto.append(info)
        title.append(record['PubmedArticle'][i]
                     ['MedlineCitation']['Article']['ArticleTitle'])

        firstname_Input = un.unidecode(authorfirst.upper()).replace('-', ' ')
        lastname_Input = un.unidecode(authorlast.upper()).replace('-', ' ')
        fullname_Input = firstname_Input+' '+lastname_Input
        first_affiliation = record['PubmedArticle'][i]['MedlineCitation']['Article']['AuthorList'][0]['AffiliationInfo']
        for k in range(len(record['PubmedArticle'][i]['MedlineCitation']['Article']['AuthorList'])):
            authors = record['PubmedArticle'][i]['MedlineCitation']['Article']['AuthorList'][k]
            # if author has an author affiliation
            if 'LastName' and 'ForeName' in authors:
                firstname_Pub = un.unidecode(
                    authors['ForeName'].upper()).replace('-', ' ')
                lastname_Pub = un.unidecode(
                    authors['LastName'].upper()).replace('-', ' ')
                fullname_Pub = firstname_Pub + ' ' + lastname_Pub

                if k == 0:
                    difference_score = difference(fullname_Pub, fullname_Input)
                    if lastname_Input != lastname_Pub:
                        difference_score += 3
                    if firstname_Input not in firstname_Pub and firstname_Pub not in firstname_Input:
                        difference_score += 3

                difference_score_new = difference(fullname_Pub, fullname_Input)
                if lastname_Input != lastname_Pub:
                    difference_score_new += 3
                if firstname_Input not in firstname_Pub and firstname_Pub not in firstname_Input:
                    difference_score_new += 3

                if difference_score >= difference_score_new:
                    #affiliation = []
                    #name = []
                    difference_score = difference_score_new

                    if authors['AffiliationInfo'] != []:
                        affiliation = [authors['AffiliationInfo'][l]['Affiliation']
                                       for l in range(len(authors['AffiliationInfo']))]
                        # elif record['PubmedArticle'][i]['MedlineCitation']['Article']['AuthorList'][0]['AffiliationInfo'] !=[]:
                    elif first_affiliation != []:
                        affiliation = ["FIRST AUTHOR AFFILIATION: " + first_affiliation[l]['Affiliation'] for l
                                       in range(len(first_affiliation))]

                    if difference_score_new == 0:
                        name = [authors['ForeName'] +
                                ' ' + authors['LastName']]
                        DS_rank = ["Same"]
                    elif difference_score_new <= 8:
                        name = [authors['ForeName'] +
                                ' ' + authors['LastName']]
                        DS_rank = ["Close"]
                    else:
                        name = [authors['ForeName'] +
                                ' ' + authors['LastName']]
                        DS_rank = ["Different"]

        dicts1[PMID[i]] = affiliation
        dicts2[PMID[i]] = name
        dicts3[PMID[i]] = DS_rank

    #dicts = pd.Series(dicts).to_frame()
    #dicts.columns = ['DOI']
    dicts1 = pd.Series(dicts1).to_frame()
    dicts1.columns = ['Affiliation']
    dicts1.Affiliation.apply(tuple)
    dicts2 = pd.Series(dicts2).to_frame()
    dicts2.columns = ['Name']
    dicts3 = pd.Series(dicts3).to_frame()
    dicts3.columns = ['DS_Rank']
    publications = dicts1.join(dicts2).join(dicts3)
    publications['Title'] = title
    publications['PeopleID'] = peopleID
    publications['Searched Term'] = matchto
    publications['Input Name'] = inputname
    if shape > 4:
        for i in range(5, shape+1):
            exec('publications[{0}Extra_{1}{0}] = extra_{1}'.format(
                "'", i-4))

    return publications

----
# CASE 1: Search with search terms

In [3]:
# pd.set_option('max_colwidth', 300)
pd.set_option('display.max_rows', 6)
df

,Row,FirstName,LastName,a,AffiliationInstitution,specialty,specialty.1,FullName,Unnamed: 8,Unnamed: 9
0,2,Chi Tim,NaN,NaN,BEST LUCK ENTERPRISES LTD T/A DR. HO CHI TIM,General Practitioner,General Practitioner|,Chi Tim Ho,NaN,32
1,6,Tak Jor,Au,NaN,DR GEORGE TAK-JOR AU,Oncology,Oncology|ONC_Colorectal Cancer Specialist|Onco...,Tak Jor Au,NaN,32
2,13,Chi Moon Peter,Chan,NaN,Dr. Chan Chi Moon Peter,Surgery,Surgery|,Chi Moon Peter Chan,NaN,32
...,...,...,...,...,...,...,...,...,...,...
102,145,Grace,Chuang,NaN,Victory A.R.T Laboratory (TST),NaN,NaN,Grace Chuang,NaN,589
103,146,Chi Tak,Chan,NaN,PRIMECARE MED. CTR.,General Practitioner,General Practitioner|,Chi Tak Chan,NaN,604
104,147,Wai Kit,Lam,NaN,HONG KONG ADVENTIST HOSPITAL - STUBBS ROAD,General Practitioner,General Practitioner|,Wai Kit Lam,NaN,633


In [4]:
import time
publications = pd.core.frame.DataFrame()
start = time.time()
placeholder = []
noresults1 = []
noresults2 = []
noresults3 = []
shape = df.shape[1]
df.iloc[:, 1:4] = df.iloc[:, 1:4].fillna('')
for row in df.itertuples():
    ID = row[1]
    authorfirst = row[2]
    authorlast = row[3]
    org = row[4]  # Taiwan

    info = row[4]

    inputname = authorfirst+' '+authorlast
    # extra
    if shape > 4:
        for i in range(5, shape+1):
            exec('extra_{}=row[{}]'.format(i-4, i))

    exec(searchterm)

    # E-utiltiies esearch searches pubmed, returns max of 500 articles
    # set retmax = 5 for Taiwan
    handle = Entrez.esearch(db="pubmed", term=search, retmax=5)
    # read parses and returns in simplified format
    record = Entrez.read(handle)
    # in the case that search results exist
    # in the case that search results exist
    if 'ErrorList' in record:
        noresults1.append(ID)
        noresults2.append(info)
        noresults3.append("No Relevant Results")

    elif record['IdList'] != []:
        # this is list of PMIDs from search
        PMID = record['IdList']
        # input PMID list of articles into efetch, returns XML of articles
        handle = Entrez.efetch(db="pubmed", id=PMID, retmode="xml")
        # parses accordingly
        record = Entrez.read(handle)
        # run function to get list of dataframes (one dataframe for each person), concat dataframe together
        publication = generator(PMID, record, shape)
        placeholder.append(publication)
    else:
        noresults1.append(ID)
        noresults2.append(info)
        noresults3.append("No Results")
if placeholder == []:
    print("No Results!!! Try Another Searched Term")
else:
    publications = pd.concat(placeholder)
end = time.time()
run_time = end-start
run_time

44.78031826019287

In [5]:
publications

,Affiliation,Name,DS_Rank,Title,PeopleID,Searched Term,Input Name,Extra_1,Extra_2,Extra_3,Extra_4,Extra_5,Extra_6
32812797,"[Department of Radiological Sciences, Universi...",[Daniel Chow],[Close],A 3D-2D Hybrid U-Net Convolutional Neural Netw...,15,,Chow,HONG KONG ADVENTIST HOSPITAL - TSUEN WAN,General Practitioner,General Practitioner|,Kwan Lung Chow,NaN,67
32584899,"[Department of Clinical Sciences, From the Cen...",[Lyndah Chow],[Close],Non-specific protection from respiratory tract...,15,,Chow,HONG KONG ADVENTIST HOSPITAL - TSUEN WAN,General Practitioner,General Practitioner|,Kwan Lung Chow,NaN,67
31036182,[Affiliation of School of Natural and Built En...,[Gregory Gnos],[Different],The development and evaluation of a microstill...,15,,Chow,HONG KONG ADVENTIST HOSPITAL - TSUEN WAN,General Practitioner,General Practitioner|,Kwan Lung Chow,NaN,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27977777,"[Genomics and Bioinformatics Centre, Malaysian...",[Keng-See Chow],[Close],Perigone Lobe Transcriptome Analysis Provides ...,15,,Chow,HONG KONG ADVENTIST HOSPITAL - TSUEN WAN,General Practitioner,General Practitioner|,Kwan Lung Chow,NaN,67
29912962,"[Epidemiology and Biostatistics, University of...",[Changwei Li],[Different],Novel genetic associations for blood pressure ...,115,,Man Lung Li,TUEN MUN HOSPITAL _ Neurology,Neurology,Neurology|NEURO_Neurologist Rx MS |Neurology,Man Lung Li,NaN,277
25663691,"[Division of Cancer Epidemiology and Genetics,...",[Jianmin Li],[Close],Excess cancers among HIV-infected people in th...,115,,Man Lung Li,TUEN MUN HOSPITAL _ Neurology,Neurology,Neurology|NEURO_Neurologist Rx MS |Neurology,Man Lung Li,NaN,277


In [6]:
if searchterm == "search='{} {}[Author]'.format(authorfirst, authorlast)":
    file_name_Out = file_name+'_Name Only'
elif searchterm == "search='({} {}[Author]) AND ({}[Affiliation])'.format(authorfirst, authorlast, org)":
    file_name_Out = file_name+'_Name With Info'

noresults = pd.DataFrame(
    {'No Results': noresults1,
     'Search Term': noresults2,
     'Reason': noresults3
     })
pprint.pprint(noresults)
# print(noresults2)

noresults.to_excel(f'{file_name_Out}_Noresults.xlsx')

#now = datetime.now()
#date_time = now.strftime("%Y%m%d")
publications.to_excel(f'{file_name_Out}_Output.xlsx')
# De empty
#Output1=pd.read_excel(f'{file_name}_Output1.xlsx', index_col = None)
# Output1_clear=Output1.drop(Output1[Output1['Name']=='[]'].index)
# Output1_clear.to_excel(f'{file_name}_Output2.xlsx')
# noresults_2 = pd.DataFrame(
#    {'No Results':list(set(list(Output1['PeopleID'][Output1[Output1['Name']=='[]'].index])+noresults1))
#    })
# pprint.pprint(noresults_2)
# noresults_2.to_excel(f'{file_name}Noresults.xlsx')

     No Results Search Term      Reason
0             2              No Results
1             6              No Results
2            13              No Results
..          ...         ...         ...
100         145              No Results
101         146              No Results
102         147              No Results

[103 rows x 3 columns]
